In [2]:
# Вероятностные алгоритмы проверки чисел на простоту

"""
Простая детерминированная проверка на простоту для небольших чисел.
Для демонстрационных целей.
"""
function simple_isprime(n::Integer)
    n ≤ 1 && return false
    n == 2 && return true
    iseven(n) && return false
    n < 9 && return true  # 3, 5, 7
    
    # Проверка делимости на нечетные числа до √n
    for i in 3:2:isqrt(n)
        if n % i == 0
            return false
        end
    end
    return true
end

"""
Тест Ферма основан на малой теореме Ферма:
Если n - простое число, то для любого a (1 < a < n) выполняется:
a^(n-1) ≡ 1 (mod n)

Если для какого-то a это сравнение не выполняется, то n - составное.
Вероятность ошибки уменьшается с увеличением количества тестов.
"""
function fermat_test(n::Integer, k::Integer=10)
    # Проверка граничных случаев
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    # k итераций теста
    for _ in 1:k
        a = rand(2:(n-2))  # Случайное основание
        if gcd(a, n) ≠ 1   # Проверка взаимной простоты
            return false
        end
        if powermod(a, n-1, n) ≠ 1  # Проверка малой теоремы Ферма
            return false
        end
    end
    return true
end

"""
Символ Якоби - обобщение символа Лежандра на составные модули.
Используется в тесте Соловэя-Штрассена для проверки критерия Эйлера.
"""
function jacobi_symbol(a::Integer, n::Integer)
    if n ≤ 0 || iseven(n)
        throw(DomainError(n, "n должно быть нечётным положительным числом"))
    end
    
    a = mod(a, n)
    result = 1
    
    while a ≠ 0
        # Удаление множителей 2 (свойство символа Якоби)
        while iseven(a)
            a ÷= 2
            mod8 = n % 8
            if mod8 == 3 || mod8 == 5
                result = -result
            end
        end
        
        # Закон квадратичной взаимности
        a, n = n, a
        
        if a % 4 == 3 && n % 4 == 3
            result = -result
        end
        
        a = mod(a, n)
    end
    
    return n == 1 ? result : 0
end

"""
Тест Соловэя-Штрассена основан на критерии Эйлера:
Для простого нечетного n и любого a (1 < a < n) выполняется:
a^((n-1)/2) ≡ (a/n) (mod n), где (a/n) - символ Якоби

Более надежен чем тест Ферма, лучше обнаруживает псевдопростые числа.
"""
function solovay_strassen_test(n::Integer, k::Integer=10)
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    for _ in 1:k
        a = rand(2:(n-2))
        
        # Проверка НОД (если НОД > 1, то n составное)
        if gcd(a, n) > 1
            return false
        end
        
        # Вычисление a^((n-1)/2) mod n
        r = powermod(a, (n-1)÷2, n)
        
        if r ≠ 1 && r ≠ n-1
            return false
        end
        
        # Вычисление символа Якоби
        j = jacobi_symbol(a, n)
        
        # Приведение символа Якоби по модулю n
        j_mod = mod(j, n)
        if j_mod < 0
            j_mod += n
        end
        
        # Проверка критерия Эйлера
        if r ≠ j_mod
            return false
        end
    end
    return true
end

"""
Тест Миллера-Рабина - наиболее надежный вероятностный тест.
Основан на том, что для простого n представление n-1 = 2^s * r
удовлетворяет одному из условий:
1) a^r ≡ 1 (mod n)
2) a^(2^j * r) ≡ -1 (mod n) для некоторого j (0 ≤ j < s)

Имеет вероятность ошибки не более 1/4^k.
"""
function miller_rabin_test(n::Integer, k::Integer=10)
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    # Представление n-1 в виде 2^s * r (r - нечетное)
    s = 0
    r = n - 1
    while iseven(r)
        r ÷= 2
        s += 1
    end
    
    # k итераций теста
    for _ in 1:k
        a = rand(2:(n-2))
        x = powermod(a, r, n)
        
        # Проверка первого условия: a^r ≡ 1 (mod n)
        if x ≠ 1 && x ≠ n-1
            j = 1
            # Проверка второго условия для j = 1..s-1
            while j ≤ s-1 && x ≠ n-1
                x = powermod(x, 2, n)
                if x == 1
                    return false  # Найден нетривиальный квадратный корень из 1
                end
                j += 1
            end
            if x ≠ n-1
                return false  # Не выполняется ни одно из условий
            end
        end
    end
    return true
end

"""
Функция для сравнительного тестирования всех алгоритмов на заданном числе.
"""
function test_primality_algorithms(n::Integer, iterations::Integer=10)
    println("Тестирование числа $n")
    println("="^50)
    
    fermat_result = fermat_test(n, iterations)
    solovay_result = solovay_strassen_test(n, iterations)
    miller_result = miller_rabin_test(n, iterations)
    deterministic_result = simple_isprime(n)
    
    println("Тест Ферма: ", fermat_result ? "Вероятно простое" : "Составное")
    println("Тест Соловэя-Штрассена: ", solovay_result ? "Вероятно простое" : "Составное")
    println("Тест Миллера-Рабина: ", miller_result ? "Вероятно простое" : "Составное")
    println("Детерминированная проверка: ", deterministic_result ? "Простое" : "Составное")
    println()
end

"""
Демонстрационная функция, показывающая работу всех алгоритмов
на различных типах чисел: простых, составных и псевдопростых.
"""
function demo()
    println("Демонстрация вероятностных алгоритмов проверки на простоту")
    println("="^60)
    println()
    
    # Тестовые числа: простые, составные и псевдопростые
    test_numbers = [17, 25, 29, 91, 97, 561, 1105, 1729]
    
    println("Тестирование на различных числах:")
    for n in test_numbers
        test_primality_algorithms(n, 5)
    end
    
    # Тестирование на известных псевдопростых числах
    println("Тестирование на псевдопростых числах (числах Кармайкла):")
    println("="^55)
    
    # Числа Кармайкла - составные числа, которые проходят тест Ферма
    carmichael_numbers = [561, 1105, 1729, 2465, 2821]
    
    for carmichael in carmichael_numbers
        println("Число Кармайкла $carmichael:")
        println("  Тест Ферма: ", fermat_test(carmichael, 10) ? "Вероятно простое" : "Составное")
        println("  Тест Соловэя-Штрассена: ", solovay_strassen_test(carmichael, 10) ? "Вероятно простое" : "Составное")
        println("  Тест Миллера-Рабина: ", miller_rabin_test(carmichael, 10) ? "Вероятно простое" : "Составное")
        println("  Детерминированная проверка: ", simple_isprime(carmichael) ? "Простое" : "Составное")
        println()
    end
    
    # Дополнительная информация о надежности алгоритмов
    println("Сравнительная характеристика алгоритмов:")
    println("="^45)
    println("• Тест Ферма: самый быстрый, но ненадежный")
    println("  Обманывается числами Кармайкла")
    println("• Тест Соловэя-Штрассена: надежнее Ферма")
    println("  Реже обманывается псевдопростыми числами") 
    println("• Тест Миллера-Рабина: наиболее надежный")
    println("  Вероятность ошибки ≤ 1/4^k")
    println("• Детерминированная проверка: точный, но медленный для больших чисел")
end

# Простой запуск демонстрации
println("Запуск демонстрации вероятностных алгоритмов проверки на простоту...")
println()

# Запускаем демо-функцию
demo()

# Дополнительные примеры использования
println()
println("Дополнительные примеры:")
println("="^30)

# Проверим несколько конкретных чисел вручную
println("Проверка числа 101 (простое):")
println("Ферма: ", fermat_test(101))
println("Соловэй-Штрассен: ", solovay_strassen_test(101))
println("Миллер-Рабин: ", miller_rabin_test(101))
println("Детерминированная: ", simple_isprime(101))
println()

println("Проверка числа 100 (составное):")
println("Ферма: ", fermat_test(100))
println("Соловэй-Штрассен: ", solovay_strassen_test(100))
println("Миллер-Рабин: ", miller_rabin_test(100))
println("Детерминированная: ", simple_isprime(100))
println()

# Простое сравнение производительности без BenchmarkTools
println("Сравнение производительности на числе 10007:")
n_test = 10007
println("Число $n_test - простое: ", simple_isprime(n_test))

# Измеряем время грубым способом
function measure_time(f, args...)
    start_time = time()
    result = f(args...)
    end_time = time()
    return result, end_time - start_time
end

println("Время выполнения (10 итераций):")
_, t1 = measure_time(fermat_test, n_test, 10)
println("  Тест Ферма: ", round(t1, digits=4), " секунд")

_, t2 = measure_time(solovay_strassen_test, n_test, 10)
println("  Тест Соловэя-Штрассена: ", round(t2, digits=4), " секунд")

_, t3 = measure_time(miller_rabin_test, n_test, 10)
println("  Тест Миллера-Рабина: ", round(t3, digits=4), " секунд")

Запуск демонстрации вероятностных алгоритмов проверки на простоту...

Демонстрация вероятностных алгоритмов проверки на простоту

Тестирование на различных числах:
Тестирование числа 17
Тест Ферма: Вероятно простое
Тест Соловэя-Штрассена: Вероятно простое
Тест Миллера-Рабина: Вероятно простое
Детерминированная проверка: Простое

Тестирование числа 25
Тест Ферма: Составное
Тест Соловэя-Штрассена: Составное
Тест Миллера-Рабина: Составное
Детерминированная проверка: Составное

Тестирование числа 29
Тест Ферма: Вероятно простое
Тест Соловэя-Штрассена: Вероятно простое
Тест Миллера-Рабина: Вероятно простое
Детерминированная проверка: Простое

Тестирование числа 91
Тест Ферма: Составное
Тест Соловэя-Штрассена: Составное
Тест Миллера-Рабина: Составное
Детерминированная проверка: Составное

Тестирование числа 97
Тест Ферма: Вероятно простое
Тест Соловэя-Штрассена: Вероятно простое
Тест Миллера-Рабина: Вероятно простое
Детерминированная проверка: Простое

Тестирование числа 561
Тест Ферма: Сост

In [3]:
# Вероятностные алгоритмы проверки чисел на простоту

"""
1. Тест Ферма
Основан на малой теореме Ферма: если n - простое, то для любого a (1 < a < n)
выполняется a^(n-1) ≡ 1 (mod n)
"""
function fermat_test(n::Integer, k::Integer=10)
    # Проверка граничных случаев
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    # k итераций теста
    for _ in 1:k
        a = rand(2:(n-2))  # Случайное основание
        if gcd(a, n) ≠ 1   # Если НОД > 1, то n составное
            return false
        end
        if powermod(a, n-1, n) ≠ 1  # Проверка малой теоремы Ферма
            return false
        end
    end
    return true
end

"""
2. Тест Соловэя-Штрассена
Основан на критерии Эйлера: для простого n выполняется
a^((n-1)/2) ≡ (a/n) (mod n), где (a/n) - символ Якоби
"""
function solovay_strassen_test(n::Integer, k::Integer=10)
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    for _ in 1:k
        a = rand(2:(n-2))
        
        # Проверка НОД (если НОД > 1, то n составное)
        if gcd(a, n) > 1
            return false
        end
        
        # Вычисление a^((n-1)/2) mod n
        r = powermod(a, (n-1)÷2, n)
        
        # Вычисление символа Якоби
        j = jacobi_symbol(a, n)
        
        # Приведение символа Якоби по модулю n
        j_mod = mod(j, n)
        if j_mod < 0
            j_mod += n
        end
        
        # Проверка критерия Эйлера
        if r ≠ j_mod
            return false
        end
    end
    return true
end

"""
Вспомогательная функция: вычисление символа Якоби
Требуется для теста Соловэя-Штрассена
"""
function jacobi_symbol(a::Integer, n::Integer)
    if n ≤ 0 || iseven(n)
        throw(DomainError(n, "n должно быть нечётным положительным числом"))
    end
    
    a = mod(a, n)
    result = 1
    
    while a ≠ 0
        # Удаление множителей 2
        while iseven(a)
            a ÷= 2
            mod8 = n % 8
            if mod8 == 3 || mod8 == 5
                result = -result
            end
        end
        
        # Закон квадратичной взаимности
        a, n = n, a
        
        if a % 4 == 3 && n % 4 == 3
            result = -result
        end
        
        a = mod(a, n)
    end
    
    return n == 1 ? result : 0
end

"""
3. Тест Миллера-Рабина
Наиболее надежный вероятностный тест. Основан на свойствах нетривиальных
квадратных корней из 1 по модулю простого числа.
"""
function miller_rabin_test(n::Integer, k::Integer=10)
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    # Представление n-1 в виде 2^s * r (r - нечетное)
    s = 0
    r = n - 1
    while iseven(r)
        r ÷= 2
        s += 1
    end
    
    # k итераций теста
    for _ in 1:k
        a = rand(2:(n-2))
        x = powermod(a, r, n)
        
        if x ≠ 1 && x ≠ n-1
            j = 1
            while j ≤ s-1 && x ≠ n-1
                x = powermod(x, 2, n)
                if x == 1
                    return false  # Найден нетривиальный квадратный корень из 1
                end
                j += 1
            end
            if x ≠ n-1
                return false
            end
        end
    end
    return true
end

"""
4. Дополнительный алгоритм: Улучшенный тест Ферма с проверкой малых делителей
Комбинирует быструю проверку малых простых делителей с тестом Ферма
"""
function enhanced_fermat_test(n::Integer, k::Integer=10)
    # Проверка граничных случаев
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    # Быстрая проверка малых простых делителей
    small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
    for p in small_primes
        if n % p == 0
            return n == p
        end
    end
    
    # k итераций теста Ферма
    for _ in 1:k
        a = rand(2:(n-2))
        if gcd(a, n) ≠ 1
            return false
        end
        if powermod(a, n-1, n) ≠ 1
            return false
        end
    end
    return true
end

"""
Функция для тестирования всех 4 алгоритмов на заданном числе
"""
function test_all_algorithms(n::Integer, iterations::Integer=10)
    println("Тестирование числа $n")
    println("="^50)
    
    # Запускаем все тесты
    fermat_result = fermat_test(n, iterations)
    solovay_result = solovay_strassen_test(n, iterations)
    miller_result = miller_rabin_test(n, iterations)
    enhanced_result = enhanced_fermat_test(n, iterations)
    
    # Выводим результаты
    println("Тест Ферма: ", fermat_result ? "Вероятно простое" : "Составное")
    println("Тест Соловэя-Штрассена: ", solovay_result ? "Вероятно простое" : "Составное")
    println("Тест Миллера-Рабина: ", miller_result ? "Вероятно простое" : "Составное")
    println("Улучшенный тест Ферма: ", enhanced_result ? "Вероятно простое" : "Составное")
    println()
    
    return (fermat_result, solovay_result, miller_result, enhanced_result)
end

"""
Демонстрация работы всех 4 алгоритмов
"""
function demo()
    println("ДЕМОНСТРАЦИЯ 4 АЛГОРИТМОВ ПРОВЕРКИ НА ПРОСТОТУ")
    println("="^55)
    println()
    
    # Тестовые наборы чисел
    small_primes = [17, 29, 97, 101]
    composites = [25, 91, 121, 1001]
    carmichael_numbers = [561, 1105, 1729]  # Псевдопростые числа
    
    println("МАЛЕНЬКИЕ ПРОСТЫЕ ЧИСЛА:")
    println("-" ^ 30)
    for n in small_primes
        test_all_algorithms(n, 5)
    end
    
    println("СОСТАВНЫЕ ЧИСЛА:")
    println("-" ^ 30)
    for n in composites
        test_all_algorithms(n, 5)
    end
    
    println("ЧИСЛА КАРМАЙКЛА (псевдопростые):")
    println("-" ^ 40)
    for n in carmichael_numbers
        test_all_algorithms(n, 10)
    end
    
    
end

# Запуск демонстрации
println("ЗАПУСК ДЕМОНСТРАЦИИ 4 АЛГОРИТМОВ ПРОВЕРКИ НА ПРОСТОТУ")
println()
demo()


special_numbers = [
    (7919, "Простое > 1000"),
    (104729, "Простое > 100000"),
    (10007, "Простое"),
    (99999, "Составное")
]

for (num, desc) in special_numbers
    println("$desc ($num):")
    test_all_algorithms(num, 5)
end

# Проверка на граничных значениях
println("ГРАНИЧНЫЕ ЗНАЧЕНИЯ:")
println("="^30)
boundary_cases = [1, 2, 3, 4]
for n in boundary_cases
    println("Число $n:")
    println("Тест Ферма: ", fermat_test(n) ? "Вероятно простое" : "Составное")
    println("Тест Соловэя-Штрассена: ", solovay_strassen_test(n) ? "Вероятно простое" : "Составное")
    println("Тест Миллера-Рабина: ", miller_rabin_test(n) ? "Вероятно простое" : "Составное")
    println("Улучшенный тест Ферма: ", enhanced_fermat_test(n) ? "Вероятно простое" : "Составное")
    println()
end

ЗАПУСК ДЕМОНСТРАЦИИ 4 АЛГОРИТМОВ ПРОВЕРКИ НА ПРОСТОТУ

ДЕМОНСТРАЦИЯ 4 АЛГОРИТМОВ ПРОВЕРКИ НА ПРОСТОТУ

МАЛЕНЬКИЕ ПРОСТЫЕ ЧИСЛА:
------------------------------
Тестирование числа 17
Тест Ферма: Вероятно простое
Тест Соловэя-Штрассена: Вероятно простое
Тест Миллера-Рабина: Вероятно простое
Улучшенный тест Ферма: Вероятно простое

Тестирование числа 29
Тест Ферма: Вероятно простое
Тест Соловэя-Штрассена: Вероятно простое
Тест Миллера-Рабина: Вероятно простое
Улучшенный тест Ферма: Вероятно простое

Тестирование числа 97
Тест Ферма: Вероятно простое
Тест Соловэя-Штрассена: Вероятно простое
Тест Миллера-Рабина: Вероятно простое
Улучшенный тест Ферма: Вероятно простое

Тестирование числа 101
Тест Ферма: Вероятно простое
Тест Соловэя-Штрассена: Вероятно простое
Тест Миллера-Рабина: Вероятно простое
Улучшенный тест Ферма: Вероятно простое

СОСТАВНЫЕ ЧИСЛА:
------------------------------
Тестирование числа 25
Тест Ферма: Составное
Тест Соловэя-Штрассена: Составное
Тест Миллера-Рабина: Составное

In [4]:
# Вероятностные алгоритмы проверки чисел на простоту

"""
1. Тест Ферма
Основан на малой теореме Ферма: если n - простое, то для любого a (1 < a < n)
выполняется a^(n-1) ≡ 1 (mod n)
"""
function fermat_test(n::Integer, k::Integer=10)
    # Проверка граничных случаев
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    # k итераций теста
    for _ in 1:k
        a = rand(2:(n-2))  # Случайное основание
        if gcd(a, n) ≠ 1   # Если НОД > 1, то n составное
            return false
        end
        if powermod(a, n-1, n) ≠ 1  # Проверка малой теоремы Ферма
            return false
        end
    end
    return true
end

"""
2. Тест Соловэя-Штрассена
Основан на критерии Эйлера: для простого n выполняется
a^((n-1)/2) ≡ (a/n) (mod n), где (a/n) - символ Якоби
"""
function solovay_strassen_test(n::Integer, k::Integer=10)
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    for _ in 1:k
        a = rand(2:(n-2))
        
        # Проверка НОД (если НОД > 1, то n составное)
        if gcd(a, n) > 1
            return false
        end
        
        # Вычисление a^((n-1)/2) mod n
        r = powermod(a, (n-1)÷2, n)
        
        # Вычисление символа Якоби
        j = jacobi_symbol(a, n)
        
        # Приведение символа Якоби по модулю n
        j_mod = mod(j, n)
        if j_mod < 0
            j_mod += n
        end
        
        # Проверка критерия Эйлера
        if r ≠ j_mod
            return false
        end
    end
    return true
end

"""
Вспомогательная функция: вычисление символа Якоби
Требуется для теста Соловэя-Штрассена
"""
function jacobi_symbol(a::Integer, n::Integer)
    if n ≤ 0 || iseven(n)
        throw(DomainError(n, "n должно быть нечётным положительным числом"))
    end
    
    a = mod(a, n)
    result = 1
    
    while a ≠ 0
        # Удаление множителей 2
        while iseven(a)
            a ÷= 2
            mod8 = n % 8
            if mod8 == 3 || mod8 == 5
                result = -result
            end
        end
        
        # Закон квадратичной взаимности
        a, n = n, a
        
        if a % 4 == 3 && n % 4 == 3
            result = -result
        end
        
        a = mod(a, n)
    end
    
    return n == 1 ? result : 0
end

"""
3. Тест Миллера-Рабина
Наиболее надежный вероятностный тест. Основан на свойствах нетривиальных
квадратных корней из 1 по модулю простого числа.
"""
function miller_rabin_test(n::Integer, k::Integer=10)
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    # Представление n-1 в виде 2^s * r (r - нечетное)
    s = 0
    r = n - 1
    while iseven(r)
        r ÷= 2
        s += 1
    end
    
    # k итераций теста
    for _ in 1:k
        a = rand(2:(n-2))
        x = powermod(a, r, n)
        
        if x ≠ 1 && x ≠ n-1
            j = 1
            while j ≤ s-1 && x ≠ n-1
                x = powermod(x, 2, n)
                if x == 1
                    return false  # Найден нетривиальный квадратный корень из 1
                end
                j += 1
            end
            if x ≠ n-1
                return false
            end
        end
    end
    return true
end

"""
4. Улучшенный тест Ферма с проверкой малых делителей
Комбинирует быструю проверку малых простых делителей с тестом Ферма
"""
function enhanced_fermat_test(n::Integer, k::Integer=10)
    # Проверка граничных случаев
    if n ≤ 3
        return n == 2 || n == 3
    end
    if iseven(n)
        return false
    end
    
    # Быстрая проверка малых простых делителей
    small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
    for p in small_primes
        if n % p == 0
            return n == p
        end
    end
    
    # k итераций теста Ферма
    for _ in 1:k
        a = rand(2:(n-2))
        if gcd(a, n) ≠ 1
            return false
        end
        if powermod(a, n-1, n) ≠ 1
            return false
        end
    end
    return true
end

"""
Функция для тестирования всех 4 алгоритмов на заданном числе
"""
function test_all_algorithms(n::Integer, iterations::Integer=10)
    println("Тестирование числа $n")
    println("="^50)
    
    # Запускаем все тесты
    fermat_result = fermat_test(n, iterations)
    solovay_result = solovay_strassen_test(n, iterations)
    miller_result = miller_rabin_test(n, iterations)
    enhanced_result = enhanced_fermat_test(n, iterations)
    
    # Выводим результаты
    println("Тест Ферма: ", fermat_result ? "Вероятно простое" : "Составное")
    println("Тест Соловэя-Штрассена: ", solovay_result ? "Вероятно простое" : "Составное")
    println("Тест Миллера-Рабина: ", miller_result ? "Вероятно простое" : "Составное")
    println("Улучшенный тест Ферма: ", enhanced_result ? "Вероятно простое" : "Составное")
    println()
end

# Демонстрация на двух числах
test_all_algorithms(17, 5)
test_all_algorithms(29, 5)

Тестирование числа 17
Тест Ферма: Вероятно простое
Тест Соловэя-Штрассена: Вероятно простое
Тест Миллера-Рабина: Вероятно простое
Улучшенный тест Ферма: Вероятно простое

Тестирование числа 29
Тест Ферма: Вероятно простое
Тест Соловэя-Штрассена: Вероятно простое
Тест Миллера-Рабина: Вероятно простое
Улучшенный тест Ферма: Вероятно простое

